<a href="https://colab.research.google.com/github/adityasingh1993/ArtificialneuralNetwork/blob/master/Keras_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


In [2]:
!unzip IMDB\ Dataset.csv.zip

Archive:  IMDB Dataset.csv.zip
  inflating: IMDB Dataset.csv        


In [3]:
from transformers import TFBertModel, BertTokenizerFast
import tensorflow as tf
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv('IMDB Dataset.csv')

In [5]:
df.head(1)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive


In [6]:
list_review=list(df["review"])
sentiments=list(df["sentiment"])
label=np.array([[0,1] if sentiment=="positive" else [1,0] for sentiment in sentiments])

In [7]:
reviews=df.review.to_string(index=None)

In [8]:
with open("corpus.txt","w") as f:
  f.writelines(reviews)

In [9]:
bert=TFBertModel.from_pretrained("bert-base-uncased")
tokenizer=BertTokenizerFast.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
max_length=256

In [11]:
input_word_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.dtypes.int32)
input_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.dtypes.int32)
segment_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.dtypes.int32)

In [12]:
tokens = tf.keras.layers.Input(shape=(max_length,), dtype=tf.dtypes.int32)
masks = tf.keras.layers.Input(shape=(max_length,), dtype=tf.dtypes.int32)

In [13]:
embedding_layer=bert.layers[0](tokens,masks)[0][:,0,:]

In [14]:
dense=tf.keras.layers.Dense(units=2,activation='softmax')(embedding_layer)

In [15]:
model=tf.keras.Model([tokens,masks],dense)

In [16]:
tokenized=tokenizer.batch_encode_plus(["Hi kya chal rha hai"],return_tensors="tf",max_length=max_length,truncation=True, pad_to_max_length=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [17]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [18]:
model.layers[2]

In [19]:
model.layers[2].trainable=False

In [20]:
tokenizer_reviews=tokenizer.batch_encode_plus(list_review,return_tensors="tf",max_length=max_length,truncation=True, pad_to_max_length=True)

In [21]:
train_split=int(0.8 * len(tokenizer_reviews["attention_mask"]))

In [22]:
train_tokens=tokenizer_reviews["input_ids"][:train_split]
train_masks=tokenizer_reviews["attention_mask"][:train_split]

In [23]:
train_tokens

<tf.Tensor: shape=(40000, 256), dtype=int32, numpy=
array([[ 101, 2028, 1997, ..., 2876, 1005,  102],
       [ 101, 1037, 6919, ...,    0,    0,    0],
       [ 101, 1045, 2245, ...,    0,    0,    0],
       ...,
       [ 101, 2023, 2265, ...,    0,    0,    0],
       [ 101, 1996, 2755, ..., 1996, 8562,  102],
       [ 101, 1045, 2031, ..., 2028, 1010,  102]], dtype=int32)>

In [24]:
test_tokens=tokenizer_reviews["input_ids"][train_split:]
test_masks=tokenizer_reviews["attention_mask"][train_split:]

In [25]:
train_labels=label[:train_split]
test_labels=label[train_split:]

In [ ]:
model.fit([train_tokens,train_masks],train_labels,epochs=5)

Epoch 1/5


  26/1250 [..............................] - ETA: 31:27 - loss: 1.2060 - accuracy: 0.4988